In [1]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')
import torch
torch.set_printoptions(linewidth=200)
import seaborn as sns
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

# Results loader object does everything
from ionpy.analysis import ResultsLoader
from pathlib import Path
root = Path("/storage/vbutoi/scratch/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 
# For using yaml configs.
%load_ext yamlmagic

In [2]:
%%yaml calibration_config

experiment:
  seed: 42
    
data:
  preload: True
  iters_per_epoch: 100

dataloader:
  batch_size: 1 
  num_workers: 1
  pin_memory: True 

optim: 
  _class: torch.optim.Adam
  lr: 3.0e-4
  weight_decay: 0.0 

train:
  epochs: 1000 
  eval_freq: 10 
  pretrained_dir: /storage/vbutoi/scratch/ESE/training/10_23_23_Dense_WMH 

log:
  checkpoint_freq: 5 
  root: '?'
  metrics:
    ece_loss:
      _fn: ese.experiment.metrics.ece_loss
      from_logits: True
    elm_loss:
      _fn: ese.experiment.metrics.elm_loss
      from_logits: True

loss_func: 
  _class: ionpy.loss.PixelCELoss 
  from_logits: True 
  batch_reduction: 'mean' 

<IPython.core.display.Javascript object>

In [3]:
%%yaml calibrator_model_cfg  

model:
  _class: '?'
  num_classes: 2
  image_channels: 1

<IPython.core.display.Javascript object>

In [4]:
%%yaml callbacks_cfg

callbacks:
  step:
    - ese.experiment.callbacks.ShowPredictions
  epoch:
    - ese.experiment.callbacks.WandbLogger
    - ionpy.callbacks.ETA
    - ionpy.callbacks.JobProgress
    - ionpy.callbacks.TerminateOnNaN
    - ionpy.callbacks.PrintLogged
    - ionpy.callbacks.ModelCheckpoint:
        monitor: ece_loss 
        phase: val

<IPython.core.display.Javascript object>

In [5]:
# Setup direcrtories
calibration_root = '/storage/vbutoi/scratch/ESE/calibration'

# WMH CONFIG
exp_name = '01_02_24_WMH_FIXED'

# Setup the root.
exp_root = f'{calibration_root}/{exp_name}'

# Create the ablation options
option_set = [
    {
        'log.root': [exp_root],
        'optim.lr': [3e-4],
        'model._class': ['ese.experiment.models.calibrators.Temperature_Scaling',
                         'ese.experiment.models.calibrators.Vector_Scaling',
                         'ese.experiment.models.calibrators.Dirichlet_Scaling',
                         'ese.experiment.models.calibrators.LTS'],
    },
]

In [6]:
from ese.scripts.utils import get_option_product
from ionpy.util import Config

# Assemble base config
base_cfg = Config(calibration_config).update([calibrator_model_cfg, callbacks_cfg])

# Get the configs
cfgs = get_option_product(exp_name, option_set, base_cfg)

In [7]:
len(cfgs)

4

## Running Jobs

In [8]:
from ese.experiment.experiment import run_ese_exp, PostHocExperiment

run_ese_exp(
    config=cfgs[0], 
    experiment_class=PostHocExperiment,
    gpu='0',
    run_name='debug',
    show_examples=False,
    track_wandb=True
)

/storage/vbutoi/projects/ionpy/util/libcheck.py:57: UserWarning: Using slow Pillow instead of Pillow-SIMD
  warn("Using slow Pillow instead of Pillow-SIMD")


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

/storage/vbutoi/projects/ionpy/util/libcheck.py:57: UserWarning: Using slow Pillow instead of Pillow-SIMD
  warn("Using slow Pillow instead of Pillow-SIMD")


Running PostHocExperiment("/storage/vbutoi/scratch/ESE/calibration/debug/20240102_195845-V7SZ-7600aa1224f792b414a07227dacea150")
---
callbacks:
  epoch:
  - ese.experiment.callbacks.WandbLogger
  - ionpy.callbacks.ETA
  - ionpy.callbacks.JobProgress
  - ionpy.callbacks.TerminateOnNaN
  - ionpy.callbacks.PrintLogged
  - ionpy.callbacks.ModelCheckpoint:
      monitor: ece_loss
      phase: val
data:
  preload: true
dataloader:
  batch_size: 1
  num_workers: 1
  pin_memory: true
experiment:
  seed: 42
log:
  checkpoint_freq: 5
  metrics:
    ece_loss:
      _fn: ese.experiment.metrics.ece_loss
      from_logits: true
    elm_loss:
      _fn: ese.experiment.metrics.elm_loss
      from_logits: true
  root: /storage/vbutoi/scratch/ESE/calibration/debug
  wandb_string: exp_name:01_02_24_WMH_FIXED-lr:0.0003-_class:ese.experiment.models.calibrators.Temperature_Scaling
loss_func:
  _class: ionpy.loss.PixelCELoss
  batch_reduction: mean
  from_logits: true
model:
  _class: ese.experiment.models.c

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vbutoi. Use `wandb login --relogin` to force relogin


Start epoch 0
Checkpointing with tag:last at epoch:0
ETA (0/999): 2024-01-02 20:06:29 - 00:00:00 remaining
Logged @ Epoch 0
metric          train          val
--------  -----------  -----------
ece_loss  0.000773359  0.000944342
elm_loss  0.00118331   0.00128661
loss      0.0060991    0.00744823
Checkpointing with tag:min-val-ece_loss at epoch:0
Start epoch 1
ETA (1/999): 2024-01-02 20:36:44 - 00:30:13 remaining
Logged @ Epoch 1
metric          train
--------  -----------
ece_loss  0.000830933
elm_loss  0.00119056
loss      0.00666023
Start epoch 2
ETA (2/999): 2024-01-02 20:36:54 - 00:30:21 remaining
Logged @ Epoch 2
metric         train
--------  ----------
ece_loss  0.00118119
elm_loss  0.00169741
loss      0.00794123
Start epoch 3
ETA (3/999): 2024-01-02 20:36:52 - 00:30:17 remaining
Logged @ Epoch 3
metric          train
--------  -----------
ece_loss  0.000870721
elm_loss  0.00140647
loss      0.00687687
Start epoch 4
ETA (4/999): 2024-01-02 20:36:55 - 00:30:19 remaining
Logged @

KeyboardInterrupt: 

In [ ]:
# # # FOR SUBMISSION
# from ese.experiment.experiment import submit_ese_exps, PostHocExperiment

# submit_ese_exps(
#     exp_root=exp_root,
#     experiment_class=PostHocExperiment,
#     config_list=cfgs,
#     available_gpus=['0', '1', '2', '3'],
#     track_wandb=True
# )